In [1]:
from nemosis import defaults
from nemosis import dynamic_data_compiler

print(defaults.static_tables)

['ELEMENTS_FCAS_4_SECOND', 'VARIABLES_FCAS_4_SECOND', 'Generators and Scheduled Loads', 'FCAS Providers']


In [2]:
from nemosis import static_table

raw_data_cache = r'D:\NEM_cache'
gen = static_table('Generators and Scheduled Loads', raw_data_cache)

INFO: Retrieving static table Generators and Scheduled Loads


In [3]:
gen.head()

,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,Technology Type - Primary,Technology Type - Descriptor,Aggregation,DUID
0,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,ADPBA1G
1,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Load,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,Y,ADPBA1L
2,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Hydro,Water,Renewable,Run of River,Y,ADPMH1
3,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Semi-Scheduled,Solar,Solar,Renewable,Photovoltaic Tracking Flat panel,Y,ADPPV1
4,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Solar,Solar,Renewable,Photovoltaic Flat panel,Y,ADPPV2


In [5]:
gen['Region'].unique()


array(['SA1', 'NSW1', 'VIC1', 'QLD1', 'TAS1'], dtype=object)

In [14]:
sheet_names = gen['Region'].unique()
import pandas as pd

with pd.ExcelWriter("NEM_Region_duids.xlsx") as writer:
    for sheet in sheet_names:
        pd.DataFrame().to_excel(writer, sheet_name=sheet, index=False)


In [15]:
solar_qld = gen.loc[(gen["Fuel Source - Descriptor"] == "Solar") & (gen["Region"] == "QLD1"), "DUID"].tolist()
wind_qld = gen.loc[(gen["Fuel Source - Descriptor"] == "Wind") & (gen["Region"] == "QLD1"), "DUID"].tolist()
coal_qld = gen.loc[(gen["Fuel Source - Descriptor"] == ("Black Coal" or "Brown Coal") ) & (gen["Region"] == "QLD1"), "DUID"].tolist()
natural_gas_qld = gen.loc[(gen["Fuel Source - Descriptor"] == ("Natural Gas" or "Natural Gas / Diesel" or "Natural Gas / Fuel Oil")) & (gen["Region"] == "QLD1"), "DUID"].tolist()

In [24]:
from itertools import zip_longest

file_name = "NEM_Region_duids.xlsx"

data = list(zip_longest(solar_qld, wind_qld, coal_qld, natural_gas_qld, fillvalue=None))

# Create a DataFrame with different-length lists
df_qld1 = pd.DataFrame(data, columns=["Solar DUID", "Wind DUID", "Coal DUID", "Natural Gas DUID"])


with pd.ExcelWriter(file_name, mode="a", if_sheet_exists="replace") as writer:
    df_qld1.to_excel(writer, sheet_name="QLD1", index=False)

In [26]:
import pandas as pd
from itertools import zip_longest

# Load data (assuming `gen` is a DataFrame containing all regions)
file_name = "NEM_Region_duids.xlsx"

regions = ["SA1", "NSW1", "VIC1", "QLD1", "TAS1"]
fuel_types = {
    "Solar": "Solar",
    "Wind": "Wind",
    "Coal": ["Black Coal", "Brown Coal"],
    "Natural Gas": ["Natural Gas", "Natural Gas / Diesel", "Natural Gas / Fuel Oil"]
}

# Dictionary to store data for all regions
region_data = {}

for region in regions:
    solar = gen.loc[(gen["Fuel Source - Descriptor"] == fuel_types["Solar"]) & (gen["Region"] == region), "DUID"].tolist()
    wind = gen.loc[(gen["Fuel Source - Descriptor"] == fuel_types["Wind"]) & (gen["Region"] == region), "DUID"].tolist()
    coal = gen.loc[gen["Fuel Source - Descriptor"].isin(fuel_types["Coal"]) & (gen["Region"] == region), "DUID"].tolist()
    natural_gas = gen.loc[gen["Fuel Source - Descriptor"].isin(fuel_types["Natural Gas"]) & (gen["Region"] == region), "DUID"].tolist()

    # Store extracted data
    data = list(zip_longest(solar, wind, coal, natural_gas, fillvalue=None))
    df = pd.DataFrame(data, columns=["Solar DUID", "Wind DUID", "Coal DUID", "Natural Gas DUID"])

    region_data[region] = df

# Write to Excel file
with pd.ExcelWriter(file_name, mode="a", if_sheet_exists="replace") as writer:
    for region, df in region_data.items():
        df.to_excel(writer, sheet_name=region, index=False)
